# Installating Octave

In [ ]:
import os
import sys
import io
import re

In [ ]:
import urllib.request as request  # Python 3

In [ ]:
# get latest stable release download link 
base_url = "https://ftpmirror.gnu.org/octave/windows/"
g = request.urlopen(base_url)
s = g.read().decode()
g.close;

r = r'(octave-\d\.\d\.\d-w64.zip)'
octave_version = re.findall(r,s)[-1][0:-8]
ziplink64bit = base_url+octave_version+"-w64.zip"
octave_zip_fullpath = os.path.join(os.environ["WINPYDIRBASE"], "t", octave_version+"-w64.zip")
siglink64bit = ziplink64bit+".sig" # check of pgp signature not implemented
print("Octave Version: "+octave_version)

In [ ]:
# start download
g = request.urlopen(ziplink64bit) 
with io.open(octave_zip_fullpath, 'wb') as f:
    f.write(g.read())
g.close
g = None

In [ ]:
#checking it's there
assert os.path.isfile(octave_zip_fullpath)

In [ ]:
#extract the zip archive
import zipfile
try:
    with zipfile.ZipFile(octave_zip_fullpath) as z:
        z.extractall(os.path.join(os.environ["WINPYDIRBASE"], "t"))
        print("Extracted all files")
except:
    print("Invalid file")

In [ ]:
# delete zip file
os.remove(octave_zip_fullpath)

In [ ]:
# Prepare environment variables
os.environ["OCTAVE_EXECUTABLE"] = os.path.join(os.environ["WINPYDIRBASE"], "t", octave_version+"-w64", "mingw64", "bin", "octave-cli.exe")
os.environ["OCTAVE_KERNEL_JSON"] = os.path.join(os.environ["WINPYDIR"], "share", "jupyter", "kernels", "octave", "kernel.json")

In [ ]:
# Installing iOctave via pip
!start cmd /C pip install octave_kernel

In [ ]:
# add octave environment variables to the Jupyter Environment
inp_str = r"""
rem ******************
rem handle {0} if included
rem ******************
if not exist "%WINPYDIRBASE%\t\{0}-w64" goto {0}_bad
set OCTAVE_EXECUTABLE=%WINPYDIRBASE%\t\{0}-w64\mingw64\bin\octave-cli.exe
set OCTAVE_KERNEL_JSON=%WINPYDIR%\share\jupyter\kernels\octave\kernel.json
:{0}_bad
""".format(octave_version)

# append to env.bat
with open(os.path.join(os.environ["WINPYDIRBASE"],"scripts","env.bat"), 'a') as file :
    file.write(inp_str)